In [328]:
import pandas as pd
import numpy as np
import difflib
#from math import cos, radians
from haversine import haversine, Unit
import folium

In [329]:
#INPUT CELL:
input_recipe = input("Please enter a recipe (e.g. Pasta Putanesca): ")
Lat_in = 35.2220
Lon_in = -101.8313


Please enter a recipe (e.g. Pasta Putanesca): chicken fontina


In [330]:
#look into function to load all
# inputs are a recipe file, a raw_ingredient file, an input location and input recipe search
farm_in = "USDA_file.csv"
recipe_in = "Recipe_file.csv"
farms = pd.read_csv(farm_in)
recipes = pd.read_csv(recipe_in)
recipes['title'] = recipes['title'].astype(str)
recipes['matched_ingredient'] = recipes['matched_ingredient'].astype(str)
farms['LAT'] = farms['LAT'].astype(float)
farms['LON'] = farms['LON'].astype(float)
farms['INGREDIENT']=farms['INGREDIENT'].apply(lambda x: x.lower())

In [331]:
def match_generator(x,y,z=3,u=0.1):
    a = y.title.unique().tolist()
    b = difflib.get_close_matches(x, a, n=z, cutoff=u)
    return y[y.title.isin(b)]

match_df = match_generator(input_recipe,recipes)
match_df

,Unnamed: 0,title,matched_ingredient,ingredient,original,categories
109077,109077,Chicken Tikka,seeds,teaspoon cumin seeds,3/4 teaspoon cumin seeds,"['Blender', 'Chicken', 'Ginger', 'Broil', 'Mar..."
109078,109078,Chicken Tikka,seeds,teaspoon coriander seeds,3/4 teaspoon coriander seeds,"['Blender', 'Chicken', 'Ginger', 'Broil', 'Mar..."
109079,109079,Chicken Tikka,milk,cups whole-milk yogurt,2 cups whole-milk yogurt,"['Blender', 'Chicken', 'Ginger', 'Broil', 'Mar..."
109080,109080,Chicken Tikka,garlic,garlic cloves,4 garlic cloves,"['Blender', 'Chicken', 'Ginger', 'Broil', 'Mar..."
109081,109081,Chicken Tikka,ginger root,fresh ginger,1 (1 1/2-inch) piece fresh ginger,"['Blender', 'Chicken', 'Ginger', 'Broil', 'Mar..."
109082,109082,Chicken Tikka,vegetable seeds,vegetable oil,3 tablespoons vegetable oil plus additional fo...,"['Blender', 'Chicken', 'Ginger', 'Broil', 'Mar..."
109083,109083,Chicken Tikka,limes,fresh lime juice,2 tablespoons fresh lime juice,"['Blender', 'Chicken', 'Ginger', 'Broil', 'Mar..."
109084,109084,Chicken Tikka,item not found,teaspoons salt,1 1/2 teaspoons salt,"['Blender', 'Chicken', 'Ginger', 'Broil', 'Mar..."
109085,109085,Chicken Tikka,fresh cut herbs,teaspoon ground turmeric,3/4 teaspoon ground turmeric,"['Blender', 'Chicken', 'Ginger', 'Broil', 'Mar..."
109086,109086,Chicken Tikka,item not found,teaspoon garam masala,1/2 teaspoon garam masala (Indian spice mixture),"['Blender', 'Chicken', 'Ginger', 'Broil', 'Mar..."


In [332]:
def farm_finder(a,b,Lat_1,Lon_1):
    i_list = a.matched_ingredient.unique().tolist()
    fs = b[b.INGREDIENT.isin(i_list)]
    fs.loc[:,'es'] = fs.apply(lambda x: haversine((Lat_1,Lon_1),(x.LAT, x.LON),unit='mi'), axis=1)
    return fs
 
farm_search = farm_finder(match_df,farms,Lat_in,Lon_in)

/Users/networkadministrator/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/networkadministrator/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [333]:
def my_agg(x):
    names = {
        'score': x['es'].min()}

    return pd.Series(names, index=['score'])

beta = farm_search[farm_search.groupby(['INGREDIENT'])['es'].transform(min) == farm_search['es']] \
.groupby(['INGREDIENT','LAT','LON']).apply(my_agg).reset_index().set_index(['INGREDIENT'])



def score_ex(x):
    try:
        alpha = beta.loc[x,'score']
        kappa = beta.loc[x,'LAT']
        gamma = beta.loc[x,'LON']
    except:
        alpha = np.NaN
        kappa = np.NaN
        gamma = np.NaN
    
    names = {
        'score': alpha,
        'Lat':  kappa,
        'Lon': gamma
        }

    return pd.Series(names, index=['score', 'Lat', 'Lon'])



match_2 = match_df.merge(match_df.matched_ingredient.apply(lambda s: score_ex(s)), left_index=True, right_index=True)

In [335]:
#UPDATES: grabbing original and grouping later to show all ingredients at given location

scores = match_2.groupby('title').score.agg(['mean']).sort_values('mean', ascending=True).reset_index()
winner = scores.title.iloc[0]

closest_df = match_2[match_2['title']==winner][['original','Lat','Lon']]
closest_2 = closest_df

closest_2

,original,Lat,Lon
114703,3 tablespoons olive oil,32.217942,-97.769211
114704,1 3 1/2 pound chicken,35.398675,-101.893804
114705,6 garlic cloves,34.962529,-101.895547
114706,1 1/2 teaspoon aniseed,34.572984,-103.346055
114707,1 28-ounce can Italian plum tomatoes,34.962529,-101.895547
114708,1/2 cup low-salt chicken broth,35.398675,-101.893804
114709,1/4 cup Ouzo,NaN,NaN
114710,1 tablespoon dried oregano,34.572984,-103.346055
114711,12 black brine-cured olives,32.217942,-97.769211
114712,4 ounces feta cheese,NaN,NaN


In [336]:
#UPDATES: here is the groupby

closest_df = closest_df.dropna().reset_index()[['original','Lat','Lon']].reset_index()

closest_frame = closest_df.groupby(['Lat', 'Lon']).original.apply(list).reset_index()


In [337]:
frame_in = pd.DataFrame({'original': ['Home'], 'Lat': [Lat_in], 'Lon': [Lon_in]})


In [338]:

         
m=folium.Map(location=[Lat_in,Lon_in])
for item in range(len(closest_frame)):
    df_new = pd.concat([closest_frame.iloc[[item]], frame_in], sort=False)
    
    points = list(df_new[['Lat','Lon']].to_records(index=False))
    #print(df_new["position_lat_degrees"].head())
    
    try:
        folium.PolyLine(locations=points,weight=5).add_to(m)
        folium.Marker([df_new["Lat"].iloc[0],df_new["Lon"].iloc[0]], 
                      popup=df_new['original'].iloc[0]).add_to(m)
        
    except: 
        print('Farm to Face Map has been Canned')
    #m.add_children(route)
m.fit_bounds([[closest_df.Lat.min(), closest_df.Lon.min()], [closest_df.Lat.max(), closest_df.Lon.max()]])
display(m)
print('The closest matching recipe is: ' + winner)
print(str(len(closest_2.dropna())) + ' of ' + str(len(closest_2)) + ' ingredients located')



The closest matching recipe is: Chicken Athena 
8 of 10 ingredients located


SyntaxError: invalid syntax (<ipython-input-269-71d525e44bb3>, line 1)